# Transformers for BoolQ Reading Comprehension

In [ ]:
# %pip install ...

## Preprocessing


## Model
### Architecture
- **Input Layer**:
    - The input to my model is the nn.Embedding layer that will be trained on the dataset with the network.
    - Each input sequence consists of a concatenated question and passage with a [SEP] token between them, marking the boundary. The separator token allows the model to distinguish between the two segments.
    - The resulting shape of the input tensor after embedding is (batch_size, sequence_length, embedding_dim).
- **6-Layer Transformer Encoder**:
    - Using the PyTorch implementation of the Transformer Encoder. The input to this model will be the output of the embedding layer with shape (batch_size, sequence_length, embedding_dim). Using six layers to learn contextual representations of the concatenated questino-passage sequence.
- **Pooling Layer**:
    - Apply *mean pooling* across the sequence length to reducing the output from (batch_size, sequence_length, embedding_dim) to (batch_size, embedding_dim). This provides a fixed-size single vector that summarizes the entire sequence for the classifier which provides the advantage of efficient memory use in training with varying sequence lengths and a fixed-sized input for my classifier.
- **2-Layer Classifier with ReLU**
    - I will implement a two-layer classifier network as defined in the project assignment. The first layer will take the output from the pooling layer of size (batch_size, embedding_dim) as its input and provide an output shape of (batch_size, hidden_dim). Using a ReLU for non-linearity. The second layer has output dimensions of (batch_size, num_classes) with num_classes=2. The output layer will use a softmax as the activation function as it is preferable over a sigmoid function for binary classification.

### Loss and Optimizer
For this binary classification task I'm using Binary Cross-Entropy Loss. BCE is widely used in binary classification problems, as it provides a probabilistic interpretation of the model's outputs, making it convenient for distinguishing between two classes. <br>
*Source*: [Binary Cross-Entropy/Log Loss for Binary Classification](https://www.geeksforgeeks.org/binary-cross-entropy-log-loss-for-binary-classification/)

For my optimizer I choose the Adam Optimizer for its adaptive learning rates and efficient handling of sparse gradients. It is well suited for deep learning tasks, provides fast convergence and has worked well in prior projects. <br>
*Source*: [Introduction to the Adam Optimizer](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

### Experiments
To tune my models' hyperparameters I will be experimenting with the following ranges:
- Learning Rate: [1e-2, 1e-7]
- 

## Evaluation


## Interpretation
